# Linear spectra of a monomer system

Importing useful classes, methods and modules:

In [ ]:
from qudofedi import Qjob, FeynmanDiagram, System, PostProcess
from qutip import sigmaz
from qiskit import Aer
import numpy as np

System parameters:

In [ ]:
e = 1.55                                     # Electronic energy gap [eV]
H = - e / 2 * sigmaz()                       # Hamiltonian of the system [eV]

mu = 1                                       # Electric dipole moment [a.u.]

system = System(H, mu)                       # System object

Spectroscopy parameters:

In [ ]:
t_min = 0.                                   # Initial time [rad/eV]
t_max = 151.                                 # Final time [rad/eV]
dt = 5.                                      # Delta time [rad/eV]
t_list = np.arange(t_min, t_max, dt)         # Time list [rad/eV]
                 
FD = FeynmanDiagram('a', t_list)             # FeynmanDiagram object

IBM Quantum backend:

In [ ]:
backend = Aer.get_backend('qasm_simulator')

In [ ]:
# from qiskit_ibm_provider import IBMProvider
# 
# provider = IBMProvider(instance='ibm-q/open/main')
# backend = provider.get_backend('ibmq_manila')

Create the Qjob object and getting the response function:

In [ ]:
job = Qjob(system, FD)                       # Qjob object

response = job.run(backend,
                   shots = 4000,
                   save_Qjob = True,
                   save_name = 'example_a_TLS',
                   save_checkpoint = True,
                  )                          # Run the simulation returning a numpy.ndarray with the results

Post-processing the results (rotating frame, exponential damping, zero-padding) to obtain the time oscillation and frequency spectra:

In [ ]:
PostProcess(job,
            RF_freq = 1.505,
            damping_rate = 29.54 * 10**(-3),
            pad = 3,
           )                                 # Post-process and print the results